## Desription

-  Images are categorized based on the emotion shown in the facial expressions (happiness, neutral, sadness, anger, surprise, disgust, fear).
-  We need to design a CNN which will classify images based on followinf facial expression 

In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

# defining dir variables 
root_dir = './data/'
train_dir = root_dir + 'train/'
test_dir = root_dir + 'test/'

In [ ]:
target_var = os.listdir(train_dir)
target_var

In [ ]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import warnings
from torch import tensor

class EmotionDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = directory
        self.transform = transform
        self.images = []
        self.labels = []
        self.label_map = {label: index for index, label in enumerate(sorted(os.listdir(directory)))}
        # Load all images and labels
        for label in sorted(os.listdir(directory)):
            label_path = os.path.join(directory, label)
            if os.path.isdir(label_path):
                for file in os.listdir(label_path):
                    file_path = os.path.join(label_path, file)
                    self.images.append(file_path)
                    self.labels.append(self.label_map[label])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx])
        image = image.convert('RGB')  # Convert image to RGB
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, tensor(label)

# Define transformations
transform = transforms.Compose([
    transforms.Lambda(lambda x: x.convert('RGB')),  # Ensure images are RGB
    transforms.Resize((224, 224)),  # Adjust size to match ViT input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Create dataset and dataloader
train_dataset = EmotionDataset(train_dir, transform=transform)
test_dataset = EmotionDataset(test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
from collections import Counter

label_map = {index: label for label, index in train_dataset.label_map.items()}

# Convert tensor labels to integers before mapping
train_labels = [label_map[label.item()] for _, label in train_dataset]
test_labels = [label_map[label.item()] for _, label in test_dataset]

train_counts = Counter(train_labels)
test_counts = Counter(test_labels)

In [ ]:
import matplotlib.pyplot as plt

# Plotting
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
ax[0].bar(train_counts.keys(), train_counts.values())
ax[0].set_title('Training Set Class Distribution')
ax[0].set_xlabel('Emotion')
ax[0].set_ylabel('Count')
ax[0].set_xticks(range(len(train_counts)))
ax[0].set_xticklabels(list(train_counts.keys()), rotation=45)

ax[1].bar(test_counts.keys(), test_counts.values())
ax[1].set_title('Testing Set Class Distribution')
ax[1].set_xlabel('Emotion')
ax[1].set_ylabel('Count')
ax[1].set_xticks(range(len(test_counts)))
ax[1].set_xticklabels(list(test_counts.keys()), rotation=45)

plt.tight_layout()
plt.show()


##### Prepare the Model for Training

In [ ]:
import os
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm.auto import tqdm
from transformers import ViTForImageClassification, AdamW

# Load a pre-trained model
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k')
num_classes = len([name for name in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, name))])

# Replace the classifier layer to match the number of classes
model.classifier = torch.nn.Linear(model.config.hidden_size, num_classes)

# Define the Training Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

# training loop
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc="Training"):
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device) if isinstance(labels, torch.Tensor) else torch.tensor(labels).to(device)

        # Reset gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.logits, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    return total_loss / len(dataloader)

epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    loss = train(model, train_loader, optimizer, criterion, device)
    print(f"Training loss: {loss}")

##### EDA

In [ ]:
def evaluate(model, dataloader):
    model.eval()
    total_correct = 0
    total_images = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.logits, 1)
            total_correct += (predicted == labels.to(device)).sum().item()
            total_images += labels.size(0)
    accuracy = total_correct / total_images
    return accuracy

accuracy = evaluate(model, test_loader)
print(f'Test Accuracy: {accuracy}')

Emotion : happy
	Training : 7215 - 3270 are male, 3945 are female
	 Testing : 1774 - 835 are male, 939 are female
Emotion : sad
	Training : 4830 - 2530 are male, 2300 are female
	 Testing : 1247 - 666 are male, 581 are female
Emotion : fearful
	Training : 4097 - 2071 are male, 2026 are female
	 Testing : 1024 - 499 are male, 525 are female
Emotion : neutral
	Training : 4965 - 3007 are male, 1958 are female
	 Testing : 1233 -  743 are male, 490 are female
Emotion : angry
	Training : 3995 - 2818 are male, 1177 are female
	 Testing : 958 - 662 are male, 296 are female
Emotion : disgusted
	Training : 436 - 246 are male, 190 are female
	 Testing : 111 - 77 are male, 34 are female
Emotion : surprised
	Training : 3171 - 1576 are male, 1595 are female
	 Testing : 831 - 446 are male, 385 are female